# Создание дашборда по пользовательским событиям для агрегатора новостей

**Описание проекта**

Яндекс. Дзен проводит анализ пользовательского взаимодействия с карточками статей. Каждую карточку определяют ее тема и источник, у которого также есть тема. Пользователей системы характеризует возрастная категория. Для менеджеров нужно сделать дашборд на основе файла выгруженных данных dash_visits.csv. 

**Задача проекта**

Используя данные Яндекс.Дзена построить дашборд с метриками взаимодействия пользователей с карточками статей. Дашборд должен помочь ответить на следующие вопросы:
- Cколько взаимодействия пользователей с карточками происходит в системе с разбивкой по темам карточек?
- Как много карточек генерируют источники с разными темами?
- Как соотносятся темы карточек и темы источников?

**План работы**
- Подключиться к базе
- Выгрузить файл с данными для дашборда
- В Tableau Public на основе выгруженного файла сформировать дашборд в соответствии с утвержденным макетом
- Ответить на поставленные вопросы
- Подготовить презентацию с отчетом: ответы на вопросы и графики

## Подключение к базе

In [1]:
!pip install psycopg2

Создаем соединение с базой:

In [2]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

db_config = {'user': 'praktikum_student', # имя пользователя
            'pwd': 'Sdf4$2;d-d30pp', # пароль
            'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
            'port': 6432, # порт подключения
            'db': 'data-analyst-zen-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                db_config['pwd'],
                                                db_config['host'],
                                                db_config['port'],
                                                db_config['db'])

engine = create_engine(connection_string)

## Загрузка файла для создания дашборда

Загружаем данные из таблицы dash_visits в файл dash_visits.csv

In [3]:
#запрос к базе
query = ''' SELECT * 
        FROM dash_visits
    '''
dash_visits = pd.io.sql.read_sql(query, con = engine) 
dash_visits.head(5)

,record_id,item_topic,source_topic,age_segment,dt,visits
0,1040597,Деньги,Авто,18-25,2019-09-24 18:32:00,3
1,1040598,Деньги,Авто,18-25,2019-09-24 18:35:00,1
2,1040599,Деньги,Авто,18-25,2019-09-24 18:54:00,4
3,1040600,Деньги,Авто,18-25,2019-09-24 18:55:00,17
4,1040601,Деньги,Авто,18-25,2019-09-24 18:56:00,27


В таблице 6 колонок:
- record_id - номер записи
- item_topic - тема карточки
- source_topic - тема источника (карточки, с которой осуществляется переход на другую карточу)
- age_segment - возрастная категория пользователя
- dt - дата и время события
- visits - количество событий

In [4]:
#считаем общее число событий
print('Всего событий:', dash_visits['visits'].sum())

Всего событий: 310207


In [5]:
#определим топ-5 карточек, с которыми было больше всего взаимодействий
df = dash_visits.groupby('item_topic').sum().sort_values(by='visits', ascending=False)
df.head(5)

,record_id,visits
item_topic,,
Наука,1587030025,21736
Отношения,1624239360,20666
Интересные факты,1608910250,19942
Общество,1501587207,19640
Подборки,1541821736,17772


In [6]:
sum = df.head(5)
sum['visits'].sum()

99756

In [7]:
#определяем временной интервал
print('Начало периода:', dash_visits['dt'].min())
print('Окончание периода:', dash_visits['dt']. max())

Начало периода: 2019-09-24 18:28:00
Окончание периода: 2019-09-24 19:00:00


Данные за один день 24.09.2019 за полчаса с 18:28 до 19:00

In [8]:
#выгрузка данных в файл dash_visits.csv
dash_visits.to_csv('dash_visits.csv', index=False)

## Формирование дашборда

Ссылка на дашборд:https://public.tableau.com/app/profile/olga.ts8460/viz/sheet4_16506187123000/Dzen?publish=yes

## Ответы на поставленные вопросы

1.Cколько взаимодействия пользователей с карточками происходит в системе с разбивкой по темам карточек?
- Максимальное количество событий наблюдается по следующим темам карточек: Наука, Отношения, Интересные факты, Общество, Подборки. 
2.Как много карточек генерируют источники с разными темами?
- Топ-5 источников: Cемейные отношения (33309 событий - 10,74%), Россия (29831 событие - 9,62%), Полезные советы (27 412 событий - 8,84%), Путешествия (24 124 события - 7,78%) и Знаменитости (23 945 событий - 7,72%). 
3.Как соотносятся темы карточек и темы источников?
 - Максимальное количество событий по темам карточек из источников: Рассказы - Путешествия (4 587 событий), Общество-Россия (3 471 событие), Наука - Кино (3 279 событие), Россия - Россия (2 847 событий), Отношения/Подборки - Полезные советы, История - Путешествия, Общество - Семейные отношения (от 2 643 до 2 795 событий)

## Подготовка презентации

Ccылка на презентацию:https://disk.yandex.ru/i/7csJS8EWljm0KQ